In [22]:
import requests
import json, os
import logging
from time import sleep
from datetime import datetime

# data AROME from opendatasoft

In [23]:
# ============
#  Parameters
# ============
dataset = 'arome-0025-sp1_sp2'
commune = 'Marseille'
lat_lon = (43.3, 5.4)

save_dir = 'data'

wait_hours = 3

In [24]:
logging.basicConfig(filename=os.path.join(save_dir, 'log.txt'),
                    format='%(asctime)s - %(levelname)s  %(message)s',
                    level=logging.INFO)

In [18]:
try:
    os.mkdir(save_dir)
    print('mkdir', save_dir)
except FileExistsError:
    pass

In [25]:
logging.info('start script')

In [27]:
def query(commune, lat_lon):
    url = "https://public.opendatasoft.com/api/records/1.0/search/"

    lat, lon = lat_lon
    params = {'dataset':dataset,
              'rows':100,
              'facet':['communes', 'code_commune'],
              'refine.commune':commune,
              'geofilter.distance':f"{lat:.2f}, {lon:.2f}, 100",
              'sort':'forecast'}

    r = requests.get(url, params=params)
    
    if r.status_code != 200:
        logging.error('query error')
        print('query error')
        print(r.text)
        data = None
        
    else:
        data = r.json()
        
    return data

In [30]:
def save_data(data):
    records = data['records']
    fields = [r['fields'] for r in records]

    times = [(f['timestamp'], f['forecast']) for f in fields]
    date_of_the_run = min(times, key=lambda x:x[1])[0]

    filename = os.path.join(save_dir, f'{date_of_the_run}.json')
    with open(filename, 'w') as outfile:
        json.dump(data, outfile)

    print(' > save:', filename)
    logging.info(f'save - run:{date_of_the_run} {len(times)} points')

In [31]:
save_data(data)

 > save: data/2020-07-28T12:00:00+00:00.json


In [29]:
data = query(commune, lat_lon)

In [21]:
while True:
    query()
    print(datetime.now(), "waiting...")
    sleep(wait_hours *60*60)

https://public.opendatasoft.com/api/records/1.0/search/?dataset=arome-0025-sp1_sp2&rows=100&facet=communes&facet=code_commune&refine.commune=Marseille&geofilter.distance=43.30%2C+5.40%2C+100&sort=forecast
 > save: data/2020-07-28T03:00:00+00:00.json
2020-07-28 18:28:56.360203 waiting...


KeyboardInterrupt: 

In [33]:
records = [r['fields'] for r in data['records']]

In [34]:
records

[{'surface_net_thermal_radiation': -17818192.0,
  'downward_short_wave_radiation_flux': 39624948.0,
  '2_metre_temperature': 26.751611000000025,
  'dist': '0.0',
  'minimum_temperature_at_2_metres': 26.630517999999995,
  'surface_net_solar_radiation': 35046400.0,
  'timestamp': '2020-07-28T12:00:00+00:00',
  'surface_latent_heat_flux': -1406048.0,
  'total_water_precipitation': 0.0,
  'relative_humidity': 64.820251,
  'surface_sensible_heat_flux': -18649744.0,
  'position': [43.3, 5.4],
  'wind_speed': 1.241419,
  'forecast': '2020-07-30T06:00:00+00:00',
  'maximum_temperature_at_2_metres': 27.408838000000003},
 {'surface_net_thermal_radiation': -17507744.0,
  'downward_short_wave_radiation_flux': 39196352.0,
  '2_metre_temperature': 27.095361000000025,
  'dist': '0.0',
  'minimum_temperature_at_2_metres': 27.03335000000004,
  'surface_net_solar_radiation': 34674076.0,
  'timestamp': '2020-07-28T12:00:00+00:00',
  'surface_latent_heat_flux': -1400384.0,
  'total_water_precipitation': 0